### 不同的memory工具
- 利用内存实现短时记忆
- 利用entity memory构建实体记忆
- 利用知识图谱来构建记忆
- 利用对话摘要来兼容内存中的长对话
- 利用token来刷新内存缓冲区
- 使用向量数据库实现长时记忆

In [1]:
# 短时记忆
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类。")
memory.chat_memory.add_ai_message("你好，我是AI。有什么可以帮助你的吗？")

memory.load_memory_variables({})  # 加载内存变量


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9852\579743416.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: 你好，我是人类。\nAI: 你好，我是AI。有什么可以帮助你的吗？'}

In [2]:
# 实现一个最近的对话窗口，超过窗口条数的对话将被删除
from langchain.memory import ConversationBufferWindowMemory

window_memory = ConversationBufferWindowMemory(k=1)

window_memory.save_context({"input": "你好，我是人类。"}, {"output": "你好，我是AI。有什么可以帮助你的吗？"})
window_memory.save_context({"input": "你好，我想吃鸡肉。"}, {"output": "你好，我是AI。我帮你找找鸡肉的做法。"})

window_memory.load_memory_variables({})  # 加载内存变量

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9852\1681470795.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=1)


{'history': 'Human: 你好，我想吃鸡肉。\nAI: 你好，我是AI。我帮你找找鸡肉的做法。'}

In [4]:
# 构建记忆实体概念清单
from langchain_community.llms import Tongyi
from langchain.memory import ConversationEntityMemory

llm = Tongyi(model="qwen-plus", temperature=0, max_tokens=1000)

entity_memory= ConversationEntityMemory(llm=llm)

_input = {
    "input": "胡八一和王胖子雪莉·杨在一起去探险，合称盗墓铁三角。",
}

# entity_memory.load_memory_variables(_input)  # 加载内存变量
entity_memory.save_context(_input, 
                           {"output": "听起来很刺激，我也想加入他们。"})

entity_memory.load_memory_variables({"input":"铁三角是谁？"})

{'history': 'Human: 胡八一和王胖子雪莉·杨在一起去探险，合称盗墓铁三角。\nAI: 听起来很刺激，我也想加入他们。',
 'entities': {'铁三角': '', '胡八一': '', '王胖子': '', '雪莉·杨': ''}}

In [10]:
# 使用知识图谱构建记忆
from langchain.memory import ConversationKGMemory
from langchain_community.llms import Tongyi

llm = Tongyi(model="qwen-plus", temperature=0, max_tokens=1000)

kg_memory = ConversationKGMemory(llm=llm)

kg_memory.save_context(
    {"input": "帮我找一下tomie。"},
    {"output": "对不起，请问什么是tomie？"}
)

kg_memory.save_context(
    {"input": "tomie是一个日本漫画角色。"},
    {"output": "好的，我知道了。请问有什么具体问题吗？"}
)

In [11]:
kg_memory.load_memory_variables({"input": "tomie是谁？"})

{'history': 'On tomie: tomie is a Japanese comic character.'}

In [12]:
kg_memory.get_current_entities("tomie最喜欢什么")

['tomie', '日本']

In [13]:
kg_memory.get_knowledge_triplets("tomie最喜欢cosplay")

[KnowledgeTriple(subject='tomie', predicate='最喜欢', object_='cosplay')]

In [14]:
# 长对话在内存中的处理方式：总结摘要以及token限制

from langchain.memory import ConversationSummaryMemory
from langchain_community.llms import Tongyi

llm = Tongyi(model="qwen-plus", temperature=0, max_tokens=1000)

summary_memory = ConversationSummaryMemory(llm=llm)

summary_memory.save_context(
    {"input": "帮我找一下tomie。"},
    {"output": "对不起，请问什么是tomie？"}
)

summary_memory.save_context(
    {"input": "tomie是一个日本漫画角色。"},
    {"output": "好的，我知道了。请问有什么具体问题吗？"}
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9852\3800426642.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm)


In [15]:
summary_memory.load_memory_variables({})

{'history': 'The human asks for help in finding "tomie." The AI initially does not recognize the term but then learns that tomie is a Japanese comic character. The AI then asks for clarification on the specific issue or question related to tomie.'}

In [17]:
messages=summary_memory.chat_memory.messages

# print(messages)

summary_memory.predict_new_summary(messages,"")

'The human asks the AI to find "tomie." The AI does not know what it is at first, but after the human explains that tomie is a Japanese comic character, the AI acknowledges and seeks clarification on any specific questions about tomie.'

In [20]:
# 使用ChatMessageHistory来快速获得对话摘要

from langchain.memory import ChatMessageHistory
from langchain_community.llms import Tongyi

llm = Tongyi(model="qwen-plus", temperature=0, max_tokens=1000)

history= ChatMessageHistory()
history.add_user_message("你好，我是人类。")
history.add_ai_message("你好，我是AI。有什么可以帮助你的吗？")

# memory= ConversationSummaryMemory.from_messages(
#     llm=llm, 
#     chat_memory=history, 
#     return_messages=True
# )

memory= ConversationSummaryMemory(
    llm=llm, 
    chat_memory=history, 
    return_messages=True,
    buffer="The human greets the AI and introduces themselves as a human. The AI responds with a greeting and offers to help the human."
)

In [21]:
# memory.buffer
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human greets the AI and introduces themselves as a human. The AI responds with a greeting and offers to help the human.', additional_kwargs={}, response_metadata={})]}

In [31]:
# 当对话持续进行且对话内容很多的时候，可以使用ConversationSummaryBufferMemory来存储对话摘要
# 这是一种非常有用的方式，他会根据token限制来自动生成对话摘要，并且在对话内容超过token限制时，自动进行摘要
# 在缓冲区内，会保留最近的k条对话内容
# 比较久的对话会被删除，在删除前会进行柴窑

from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.chat_models.tongyi import ChatTongyi

llm = ChatTongyi(model="qwen-plus", temperature=0, max_tokens=1000)

summary_buffer_memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=10, # 调整此值
    return_messages=True,
)

summary_buffer_memory.save_context(
    {"input": "帮我找一下tomie。"},
    {"output": "对不起，请问什么是tomie？"}
)

summary_buffer_memory.save_context(
    {"input": "tomie是一个日本漫画角色。"},
    {"output": "好的，我知道了。请问有什么具体问题吗？"}
)

summary_buffer_memory.save_context(
    {"input": "他最近打算cos旺财。"},
    {"output": "好的，我知道了。需要旺财的信息吗？"}
)

In [32]:
summary_buffer_memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks for help in finding "tomie," and the AI initially does not know what it refers to. The human explains that Tomie is a Japanese comic character, and the AI acknowledges this, asking if there is a specific question about Tomie. Later, the human mentions they plan to cosplay as Wang Cai, and the AI asks if they need information about Wang Cai.', additional_kwargs={}, response_metadata={})]}

In [33]:
# 使用token长度决定何时刷新内存
from langchain.memory import ConversationTokenBufferMemory
from langchain_community.chat_models.tongyi import ChatTongyi

llm = ChatTongyi(model="qwen-plus", temperature=0, max_tokens=1000)

token_buffer_memory = ConversationTokenBufferMemory(
    llm=llm, 
    max_token_limit=100,  # 调整此值
)

token_buffer_memory.save_context(
    {"input": "帮我找一下tomie。"},
    {"output": "对不起，请问什么是tomie？"}
)

token_buffer_memory.save_context(
    {"input": "tomie是一个日本漫画角色。"},
    {"output": "好的，我知道了。请问有什么具体问题吗？"}
)

token_buffer_memory.save_context(
    {"input": "他最近打算cos旺财。"},
    {"output": "好的，我知道了。需要旺财的信息吗？"}
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9852\1604842732.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  token_buffer_memory = ConversationTokenBufferMemory(


In [34]:
token_buffer_memory.load_memory_variables({})

{'history': 'Human: 他最近打算cos旺财。\nAI: 好的，我知道了。需要旺财的信息吗？'}

In [5]:
# 长时记忆的实现方式
# 通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话

from langchain.embeddings import DashScopeEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

longterm_memory = ConversationBufferMemory()

longterm_memory.save_context(
    {"input": "帮我找一下tomie。"},
    {"output": "对不起，请问什么是tomie？"}
)

longterm_memory.save_context(
    {"input": "tomie是一个日本漫画角色。"},
    {"output": "好的，我知道了。请问有什么具体问题吗？"}
)

longterm_memory.save_context(
    {"input": "他最近打算cos旺财。"},
    {"output": "好的，我知道了。需要旺财的信息吗？"}
)


vectorstore = FAISS.from_texts(
    longterm_memory.buffer.split("\n"),  # 将对话内容按行分割
    embedding=DashScopeEmbeddings(),
)

FAISS.save_local(vectorstore, "test_faiss")


In [6]:
FAISS.load_local("test_faiss", 
                 DashScopeEmbeddings(),
                 allow_dangerous_deserialization=True
                 ).similarity_search(
    "tomie最近打算cos谁？"
)

[Document(id='33c79095-b447-4621-9121-73e6cd182b6f', metadata={}, page_content='Human: 帮我找一下tomie。'),
 Document(id='b493a37e-501e-4412-b98a-bfc15ece4896', metadata={}, page_content='AI: 对不起，请问什么是tomie？'),
 Document(id='f493631b-51c6-4c49-9e1f-ba17ff7bb537', metadata={}, page_content='Human: 他最近打算cos旺财。'),
 Document(id='da3165c9-030f-4989-a81c-ae48742c44dc', metadata={}, page_content='Human: tomie是一个日本漫画角色。')]

In [1]:
# 重新打开，模拟清理内存
from langchain.vectorstores import FAISS
from langchain.embeddings import DashScopeEmbeddings
from langchain.memory import VectorStoreRetrieverMemory

r1=FAISS.load_local("test_faiss",
                    DashScopeEmbeddings(),
                    allow_dangerous_deserialization=True
                    )
r2=r1.as_retriever(
    search_kwargs={"k": 1}  # 限制返回的文档数量
)

vector_memory = VectorStoreRetrieverMemory(
    retriever=r2
)

vector_memory.load_memory_variables({"input": "tomie最近打算cos谁？"})

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9520\3040828667.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  vector_memory = VectorStoreRetrieverMemory(


{'history': 'Human: 帮我找一下tomie。'}